# Impact of weak rate options

LINX includes a number of different switches and options for computing the rate of proton-neutron interconversion, which can have an impact on the prediction for the abundance of helium-4.  We explore these settings in this notebook.

## Preamble

In [1]:
%load_ext autoreload
%autoreload
import numpy as np
import jax.numpy as jnp
import jax
from jax import jit, vmap
import sys

sys.path.append("..")
import linx.const as const 
from linx.weak_rates import WeakRates
from linx.nuclear import NuclearRates
from linx.background import BackgroundModel
from linx.abundances import AbundanceModel

See background_evolution for more about how LINX computes background thermodynamics.

In [2]:
thermo_model_DNeff = BackgroundModel()

(
    t_vec_ref, a_vec_ref, rho_g_vec, rho_nu_vec, rho_NP_vec, P_NP_vec, Neff_vec 
) = thermo_model_DNeff(0.)


`\         /´  ||||        ||||  |||||     ||||  ||||   ||||
 /\_______/\   ||||        ||||  |||||||   ||||   |||| ||||
 ) __` ´__ (   ||||        ||||  |||| |||| ||||    |||||||
/  `-|_|-´  \  ||||        ||||  ||||  |||| |||    ||||||| 
/   (_x_)   \  ||||||||||  ||||  ||||   |||||||   |||| ||||
  )  `-´  (    ||||||||||  ||||  ||||    ||||||  ||||   ||||
 
Compiling thermodynamics model...


We'll use the "key" PRIMAT network here, though the prediction for helium-4 does not depend sensitively on the choice of network.

In [3]:
network = 'key_PRIMAT_2023'
abundance_model = AbundanceModel(NuclearRates(nuclear_net=network),
               weak_rates=WeakRates(RC_corr=True,thermal_corr=True,FM_corr=True, weak_mag_corr=True))

## Weak rates

By default, LINX includes the following corrections, all of which can be toggled.  These corrections are derived in https://arxiv.org/abs/1801.08023, which we'll call Pitrou_2018 here. 

- Virtual photon radiative corrections, labeled as "RC0" in Pitrou_2018. These correspond to including one-loop diagrams with a photon loop, and no emission of real photons; 
- Finite temperature radiative corrections and Bremsstrahlung corrections, labeled as "ThRC+BS" in Pitrou_2018 (we do not use separate thermal and Bremsstrahlung corrections).  These corrections are included only as a table of values obtained using PRyMordial, assuming SBBN. A proper calculation of these corrections for arbitrary background evolution histories requires quadrature in two dimensions, which cannot be straightforwardly performed in JAX. However, these corrections are known to be small within SBBN, and can be safely neglected for now;
- Finite nucleon mass corrections, labeled as "FM" in Pitrou_2018. If used in combination with virtual photon radiative corrections, we do not use separate "RC0" and "FM" corrections; we use instead corrections labeled "RC+FM" in Pitrou_2018, and
- The weak magnetism correction, denoted "WM" in Pitrou_2018.

Let's explore what happens when we pass in different combinations of options to the `WeakRates` module in LINX:

In [5]:
res_arr = []
bool_arrs = [[False, False, False, False],# Born
             [False, False, True, False], # Born + FM
             [False, False, True, True], # Born + FM + WM
             [True, False, False, False],# RC0
             [True, False, True, True],# RC + FM + WM
             [True, True, True, True]# RC + ThRC+BS + FM + WM
            ]
             
for bool_arr in bool_arrs:
    abundance_model = AbundanceModel(NuclearRates(nuclear_net=network),
                   weak_rates=WeakRates(RC_corr=bool_arr[0],thermal_corr=bool_arr[1],FM_corr=bool_arr[2], weak_mag_corr=bool_arr[3]))
    
    Yn, Yp, Yd, Yt, YHe3, Ya, YLi7, YBe7 = abundance_model(
        rho_g_vec,
        rho_nu_vec,
        jnp.zeros_like(rho_g_vec),
        jnp.zeros_like(rho_g_vec),
        t_vec = t_vec_ref,
        a_vec = a_vec_ref
    )
    res_arr = np.append(res_arr,4*Ya)

Compiling abundance model...
Compiling abundance model...
Compiling abundance model...
Compiling abundance model...
Compiling abundance model...


Like in background_evolution, the abundance model needed to compile each time for each different set of options.

We can make a table of our results for our predictions for $Y_{\rm{P}}$:

In [14]:
settings = ["Born", "Born + FM", "Born + FM + WM", "RC0", "RC + FM + WM", "RC + ThRC+BS + FM + WM"]
titles = ['Configuration','Yp']

table = []
for i in range(len(settings)):
    row = [settings[i],res_arr[i]]
    table.append(row)

col_width = 25
decimal_places = 5

print(f"{titles[0].ljust(col_width)} {titles[1].ljust(col_width)}")
print("-" * (col_width * 2))
for row in table:
    formatted_row = f"{str(row[0])[:col_width-1]:<{col_width}}" 
    for item in row[1:]:
        formatted_row += f"{item:<{col_width}.{decimal_places}f}"
    print(formatted_row)

Configuration             Yp                       
--------------------------------------------------
Born                     0.24279                  
Born + FM                0.24389                  
Born + FM + WM           0.24401                  
RC0                      0.24593                  
RC + FM + WM             0.24715                  
RC + ThRC+BS + FM + WM   0.24714                  


In [19]:
settings = ["Born", "Born + FM", "Born + FM + WM", "RC0", "RC + FM + WM", "RC + FM + WM + ThRC+BS"]
titles = ['Configuration','% Yp']

def percentage(varied,fiducial):
    return 100*np.abs((varied - fiducial)/((fiducial)))

table = []
for i in range(len(settings)):
    row = [settings[i],percentage(res_arr[i],res_arr[0])]
    table.append(row)

col_width = 25
decimal_places = 5

print(f"{titles[0].ljust(col_width)} {titles[1].ljust(col_width)}")
print("-" * (col_width * 2))
for row in table:
    formatted_row = f"{str(row[0])[:col_width-1]:<{col_width}}" 
    for item in row[1:]:
        formatted_row += f"{item:<{col_width}.{decimal_places}f}"
    print(formatted_row)

Configuration             % Yp                     
--------------------------------------------------
Born                     0.00000                  
Born + FM                0.45226                  
Born + FM + WM           0.50142                  
RC0                      1.29176                  
RC + FM + WM             1.79416                  
RC + FM + WM + ThRC+BS   1.79048                  


This last setting is the most accurate and is used by default in LINX.